In [ ]:
!pip install dgl-0.3-cp37-cp37m-manylinux1_x86_64.whl

In [ ]:
!pip uninstall -y networkx

Uninstalling networkx-2.5:
  Successfully uninstalled networkx-2.5


In [ ]:
!pip install networkx-2.5-py3-none-any.whl

Processing ./networkx-2.5-py3-none-any.whl
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.7 which is incompatible.


In [ ]:
!pip uninstall -y imgaug

Uninstalling imgaug-0.2.7:
  Successfully uninstalled imgaug-0.2.7


In [ ]:
!pip install imgaug-0.2.7-py3-none-any.whl

Processing ./imgaug-0.2.7-py3-none-any.whl
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.7 which is incompatible.


In [ ]:
!pip install tensorboardX-2.1-py2.py3-none-any.whl

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
path="/content/drive/MyDrive/Others"
#path="/content/"
os.chdir(path)
os.listdir(path)
!python utils.py
!python utils_data.py
!python utils_layers.py

In [ ]:
import argparse
import json
import os
import time

import dgl.init
import numpy as np
import tensorboardX
import torch as th
import torch.nn.functional as F

import utils_data
from utils_layers import GCNNet

In [ ]:
args={'model':'GCN_TwoLayers',
  'dataset':'wisconsin',
  'num_hidden':32,
  'num_heads_layer_one':1,
  'num_heads_layer_two':1,
  'dropout_rate':0.5,
  'learning_rate':0.05,
  'weight_decay_layer_one':5e-06,
  'weight_decay_layer_two':5e-06,
  'num_epochs_patience':100,
  'num_epochs_max':5000,
  'run_id':0,
  'dataset_split':'splits/wisconsin_split_0.6_0.2_0.npz',
  'learning_rate_decay_patience':50,
  'learning_rate_decay_factor':0.8
   }

In [ ]:
import pandas as pd

if __name__ == '__main__':
    record = pd.DataFrame(columns=['Train Loss', 'Val Loss', 'Train Acc', 'Val Acc'])
    '''
    parser = argparse.ArgumentParser()
    parser.add_argument('--dataset', type=str)
    parser.add_argument('--num_hidden', type=int)
    parser.add_argument('--num_heads_layer_one', type=int)
    parser.add_argument('--num_heads_layer_two', type=int)
    parser.add_argument('--dropout_rate', type=float)
    parser.add_argument('--learning_rate', type=float)
    parser.add_argument('--weight_decay_layer_one', type=float)
    parser.add_argument('--weight_decay_layer_two', type=float)
    parser.add_argument('--num_epochs_patience', type=int, default=100)
    parser.add_argument('--num_epochs_max', type=int, default=5000)
    parser.add_argument('--run_id', type=str)
    parser.add_argument('--dataset_split', type=str)
    parser.add_argument('--learning_rate_decay_patience', type=int, default=50)
    parser.add_argument('--learning_rate_decay_factor', type=float, default=0.8)
    args = parser.parse_args()
    vars(args)['model'] = 'GCN_TwoLayers'
    '''

    if args["dataset_split"] == 'jknet':
        g, features, labels, train_mask, val_mask, test_mask, num_features, num_labels = utils_data.load_data(
            args['dataset'], None, 0.6, 0.2)
    else:
        g, features, labels, train_mask, val_mask, test_mask, num_features, num_labels = utils_data.load_data(
            args['dataset'], args['dataset_split'])

    g.set_n_initializer(dgl.init.zero_initializer)
    g.set_e_initializer(dgl.init.zero_initializer)

    net = GCNNet(num_input_features=num_features, num_output_classes=num_labels, num_hidden=args['num_hidden'],
            dropout_rate=args['dropout_rate'],num_heads_layer_one=args['num_heads_layer_one'], num_heads_layer_two=args['num_heads_layer_two'])

    optimizer = th.optim.Adam([{'params': net.gcn1.parameters(), 'weight_decay': args['weight_decay_layer_one']},
                               {'params': net.gcn2.parameters(), 'weight_decay': args['weight_decay_layer_two']}],
                              lr=args['learning_rate'])
    learning_rate_scheduler = th.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, factor=args['learning_rate_decay_factor'],patience=args['learning_rate_decay_patience'])
    #writer = tensorboardX.SummaryWriter(logdir=f'runs/{args['model']}_{args['run_id']}')
    writer = tensorboardX.SummaryWriter(logdir=f'runs/{args["model"]}_{args["run_id"]}')

    features = features
    labels = labels
    train_mask = train_mask
    val_mask = val_mask
    test_mask = test_mask

    # Adapted from https://github.com/PetarV-/GAT/blob/master/execute_cora.py
    patience = args['num_epochs_patience']
    vlss_mn = np.inf
    vacc_mx = 0.0
    vacc_early_model = None
    vlss_early_model = None
    state_dict_early_model = None
    curr_step = 0

    # Adapted from https://docs.dgl.ai/tutorials/models/1_gnn/1_gcn.html
    dur = []
    test_time = 0.0
    for epoch in range(500):
        t0 = time.time()

        net.train()
        train_logits = net(g, features)
        train_logp = F.log_softmax(train_logits, 1)
        train_loss = F.nll_loss(train_logp[train_mask], labels[train_mask])
        train_pred = train_logp.argmax(dim=1)
        train_acc = th.eq(train_pred[train_mask], labels[train_mask]).float().mean().item()

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        net.eval()
        with th.no_grad():
            val_logits = net(g, features)
            val_logp = F.log_softmax(val_logits, 1)
            val_loss = F.nll_loss(val_logp[val_mask], labels[val_mask]).item()
            val_pred = val_logp.argmax(dim=1)
            val_acc = th.eq(val_pred[val_mask], labels[val_mask]).float().mean().item()

        learning_rate_scheduler.step(val_loss)

        dur.append(time.time() - t0)

        print(
            "Epoch {:05d} | Train Loss {:.4f} | Train Acc {:.4f} | Val Loss {:.4f} | Val Acc {:.4f} | Time(s) {:.4f}".format(
                epoch, train_loss.item(), train_acc, val_loss, val_acc, sum(dur) / len(dur)))

        writer.add_scalar('Train Loss', train_loss.item(), epoch)
        writer.add_scalar('Val Loss', val_loss, epoch)
        writer.add_scalar('Train Acc', train_acc, epoch)
        writer.add_scalar('Val Acc', val_acc, epoch)

        test_time += (sum(dur) / len(dur))
        new={'Train Loss':train_loss.item(),'Val Loss':val_loss,'Train Acc':train_acc,'Val Acc':val_acc}

        record=record.append(new,ignore_index=True)   # ignore_index=True,表示不按原来的索引，从0开始自动递增

        # Adapted from https://github.com/PetarV-/GAT/blob/master/execute_cora.py
        if val_acc >= vacc_mx or val_loss <= vlss_mn:
            if val_acc >= vacc_mx and val_loss <= vlss_mn:
                vacc_early_model = val_acc
                vlss_early_model = val_loss
                state_dict_early_model = net.state_dict()
            vacc_mx = np.max((val_acc, vacc_mx))
            vlss_mn = np.min((val_loss, vlss_mn))
            curr_step = 0
        else:
            curr_step += 1
            if curr_step >= patience:
                print("")

    #drive.mount("/content")
    
    record.to_csv('Test_Result/test_{model}_{dataset}_{rec_time}.csv'.format(model=args["model"], dataset=args["dataset"], rec_time=test_time))
    print(test_time)

    net.load_state_dict(state_dict_early_model)
    net.eval()
    with th.no_grad():
        test_logits = net(g, features)
        test_logp = F.log_softmax(test_logits, 1)
        test_loss = F.nll_loss(test_logp[test_mask], labels[test_mask]).item()
        test_pred = test_logp.argmax(dim=1)
        test_acc = th.eq(test_pred[test_mask], labels[test_mask]).float().mean().item()
        test_hidden_features = net.gcn1(g, features).cpu().numpy()

        final_train_pred = test_pred[train_mask].cpu().numpy()
        final_val_pred = test_pred[val_mask].cpu().numpy()
        final_test_pred = test_pred[test_mask].cpu().numpy()
    '''
    results_dict = vars(args)
    results_dict['test_loss'] = test_loss
    results_dict['test_acc'] = test_acc
    results_dict['actual_epochs'] = 1 + epoch
    results_dict['val_acc_max'] = vacc_mx
    results_dict['val_loss_min'] = vlss_mn
    results_dict['total_time'] = sum(dur)
    with open(os.path.join('runs', f'{args.model}_{args.run_id}_results.txt'), 'w') as outfile:
        outfile.write(json.dumps(results_dict) + '\n')
    np.savez_compressed(os.path.join('runs', f'{args.model}_{args.run_id}_hidden_features.npz'),
                        hidden_features=test_hidden_features)
    np.savez_compressed(os.path.join('runs', f'{args.model}_{args.run_id}_final_train_predictions.npz'),
                        final_train_predictions=final_train_pred)
    np.savez_compressed(os.path.join('runs', f'{args.model}_{args.run_id}_final_val_predictions.npz'),
                        final_val_predictions=final_val_pred)
    np.savez_compressed(os.path.join('runs', f'{args.model}_{args.run_id}_final_test_predictions.npz'),
                        final_test_predictions=final_test_pred)
    '''

test1
test4
test5
<class 'scipy.sparse.csr.csr_matrix'>
DGLGraph(num_nodes=251, num_edges=750,
         ndata_schemes={}
         edata_schemes={})
<class 'dgl.graph.DGLGraph'>
<class 'dgl.graph.DGLGraph'>
test_end
<class 'dgl.graph.DGLGraph'>
Epoch 00000 | Train Loss 1.6094 | Train Acc 0.2000 | Val Loss 1.5438 | Val Acc 0.5500 | Time(s) 0.0189
Epoch 00001 | Train Loss 1.5360 | Train Acc 0.3083 | Val Loss 1.4421 | Val Acc 0.5000 | Time(s) 0.0180
Epoch 00002 | Train Loss 1.4130 | Train Acc 0.4500 | Val Loss 1.3479 | Val Acc 0.5000 | Time(s) 0.0175
Epoch 00003 | Train Loss 1.3558 | Train Acc 0.5333 | Val Loss 1.3005 | Val Acc 0.5250 | Time(s) 0.0177
Epoch 00004 | Train Loss 1.2979 | Train Acc 0.4500 | Val Loss 1.2967 | Val Acc 0.5250 | Time(s) 0.0177
Epoch 00005 | Train Loss 1.2571 | Train Acc 0.4500 | Val Loss 1.3031 | Val Acc 0.5375 | Time(s) 0.0175
Epoch 00006 | Train Loss 1.2256 | Train Acc 0.5500 | Val Loss 1.2906 | Val Acc 0.5125 | Time(s) 0.0174
Epoch 00007 | Train Loss 1.1795 | T